In [1]:
import os
import sys
import pandas as pd
import numpy as np
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

In [55]:
# FRAM001P
conn = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

#crdmprd
conn2 = cx_Oracle.connect(
    user="AD21236",
    password="C#ntury123",
    dsn="racorap33-scan.corp.intranet/dwpr01p_users",
    encoding="UTF-8")

DatabaseError: ORA-28001: the password has expired

In [ ]:
%%time

query2 = """

            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    ,O.STATE
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  WIRE_CENTER_ID
                    ,STATE
                    ,count(distinct LUID) as cnt
                    
            FROM    QD
            
            GROUP BY
                    WIRE_CENTER_ID
                    ,STATE


"""

query = """

select distinct DTN, WIRE_CENTER_ID
from LQSMO.LU_CIRCUITS_O

"""


df_ora = pd.read_sql(query, con=conn) #look up for wc to dtn
df_ora2 = pd.read_sql(query2, con=conn) #contains total lu at wc level

In [ ]:
%%time

mg_query = """

            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND UNI_POLYGONID in (
                                          '1179499',  -- PHNXAZNE
                                          '1179547', '1179498', '1179121','1180067','1180494', -- PHNXAZSO
                                          '1178832', -- ARVDCOMA
                                          '1179849','1180344','1180107','1180306','1180028', '1178694', '1179569',-- MPLSMNBE 
                                          '1179862', '1179836', '1179604', '1179838', '1179887', '1180054', '1180222', -- MPLSMNBE
                                          '1178144', --PYLPWA01
                                          '1179687', '1179674','1180254', --GLDNCOMA
                                          '1180012','1180391', -- PTLDOR11
                                          '1179864', -- DNVRCOMA 
                                          '1179553','1179924', -- STPLMNBE 
                                          '1179688','1179157', '1179574',  -- MPLSMNTF
                                          '1180026','1180288','1179808','1179976', -- MPLSMNGE
                                          '1179583', -- DNVRCODC
                                          '1179774','1180265','1180031','1180237', -- CRYSMNCR
                                          '1180315','1189074', -- MRRYUTMA
                                          '1179708','1179714','1179719','1180072','1180075','1180089','1180095', -- TCSNAZMA
                                          '1180096','1179639','1179689','1179690','1179700','1179701','1179702', -- TCSNAZMA
                                          '1179703','1179704','1179718','1179793','1179803','1180092','550715', -- TCSNAZMA
                                          '1180094','1180507', --TCSNAZMA
                                          '1179641','1179914' --TCSNAZCR
                                            ) --add in MG polygon IDs here
                    
            )
            
            SELECT  WIRE_CENTER_ID
                    ,count(distinct LUID) as cnt
                    
            FROM    QD
            
            GROUP BY
                    WIRE_CENTER_ID


"""

mg_lu_ora = pd.read_sql(mg_query, con=conn) #contains total lu at wc level

In [ ]:
mg_lu_df = mg_lu_ora.rename(columns = {'CNT': 'MG_ENABLED_LU_CNT'})

In [ ]:
# Input excel MG raw data
sf_df = pd.read_excel(r'Salesforce Data/salesdata.xlsx',sheet_name = 'Ella XGS Sales') 

In [ ]:
sf_df['Purchased Data Rate'].fillna('missing_speed', inplace=True)

In [ ]:
# before import salesforce DTN field needs to be string
sf_df['DTN'] = sf_df['DTN'].apply(lambda x: '{:.0f}'.format(x))
sf_df['DTN'] = sf_df['DTN'].astype(str)

In [ ]:
# select only account status Active or Pending Activation
sf_df = sf_df[(sf_df['Account Status'] == 'Active')|(sf_df['Account Status'] == 'Pending Activation')]

In [ ]:
df_ora2.head()

In [ ]:
# this has state/wirecenter/Enabled LU counts at wire center level
df_wc_state = df_ora2.copy()

In [ ]:
#Adding wire center detail to Salesforce
combine_df = df_ora.merge(sf_df, how = 'outer', on = 'DTN') #combine now has wire center id
combine_df.head()

In [ ]:
#Adding LU count detail to Sales force combine_df
combine_df_details = df_wc_state.merge(combine_df, how = 'outer', on = 'WIRE_CENTER_ID')
combine_df_details['STATE'].value_counts() #check for nulls

In [ ]:
#Update CNT column name
combine_df_details = combine_df_details.rename(columns = {'CNT': 'Enabled_LUs_WireCenter_Level'})

In [ ]:
summary_df = combine_df_details.copy()
summary_df = summary_df.fillna(0)

In [ ]:
summary_df['Order Number'] = np.where(summary_df['Order Number'] == 0.0, np.NaN, summary_df['Order Number'])

In [ ]:
summary_df.head()

In [56]:
# check if new speeds are popping up
summary_df['Purchased Data Rate'].value_counts()

0            136386
940M/940M       768
200M/200M       319
1G/1G           124
3G/3G             6
8G/8G             1
Name: Purchased Data Rate, dtype: int64

In [57]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# summary_df['Due Date'].value_counts()

In [58]:
enablement_base_group = summary_df.groupby(['STATE','WIRE_CENTER_ID'], as_index=False)

In [59]:
# create base enablement for wire centers
enablement_base_state = enablement_base_group['Enabled_LUs_WireCenter_Level'].max()

In [60]:
from datetime import timedelta, date
from pandas.tseries.offsets import DateOffset

yesterday_date = date.today() - timedelta(days=90)
future_date = date.today() + timedelta(days=90)
# wire center level - if new speeds pop up, add to below list to create new columns
#combine2_sum_df = combine2_df.groupby(by = ['WIRE_CENTER_ID', 'Purchased Data Rate', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Work Order Number'].nunique()

# state level - get total purchases
# combine2_state_df.groupby(by = ['STATE_CD', 'Purchased Data Rate'], as_index = False)['Work Order Number'].nunique()
speed_940m = summary_df[((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Active'))&((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '940M/940M'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()
speed_940m = speed_940m.rename(columns = {'Order Number': '940M - Active Subscribers'})

# speed_940m_pending = summary_df[((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Pending Activation'))&((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '940M/940M'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()
speed_940m_pending = summary_df[((summary_df['Due Date'] == 0)| \
                                 ((pd.to_datetime(summary_df['Due Date']).dt.date > yesterday_date) & (pd.to_datetime(summary_df['Due Date']).dt.date <= future_date))) \
                                &((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Pending Activation')) \
                                &((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '940M/940M'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()

speed_940m_pending = speed_940m_pending.rename(columns = {'Order Number': '940M - Pending Activation Subscribers'})

# 200M

speed_200m = summary_df[((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Active'))&((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '200M/200M'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()
speed_200m = speed_200m.rename(columns = {'Order Number': '200M - Active Subscribers'})

speed_200m_pending = summary_df[((summary_df['Due Date'] == 0)| \
                                 ((pd.to_datetime(summary_df['Due Date']).dt.date > yesterday_date) & (pd.to_datetime(summary_df['Due Date']).dt.date <= future_date))) \
                                &((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Pending Activation')) \
                                &((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '200M/200M'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()

speed_200m_pending = speed_200m_pending.rename(columns = {'Order Number': '200M - Pending Activation Subscribers'})

# 1G

speed_1g = summary_df[((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Active'))&((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '1G/1G'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()
speed_1g = speed_1g.rename(columns = {'Order Number': '1G - Active Subscribers'})

speed_1g_pending = summary_df[((summary_df['Due Date'] == 0)| \
                                 ((pd.to_datetime(summary_df['Due Date']).dt.date > yesterday_date) & (pd.to_datetime(summary_df['Due Date']).dt.date <= future_date))) \
                              &((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Pending Activation')) \
                              &((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '1G/1G'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()

speed_1g_pending = speed_1g_pending.rename(columns = {'Order Number': '1G - Pending Activation Subscribers'})

# 3G
speed_3g = summary_df[((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Active'))&((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '3G/3G'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()
speed_3g = speed_3g.rename(columns = {'Order Number': '3G - Active Subscribers'})

speed_3g_pending = summary_df[((summary_df['Due Date'] == 0)| \
                                 ((pd.to_datetime(summary_df['Due Date']).dt.date > yesterday_date) & (pd.to_datetime(summary_df['Due Date']).dt.date <= future_date))) \
                              &((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Pending Activation')) \
                              &((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '3G/3G'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()

speed_3g_pending = speed_3g_pending.rename(columns = {'Order Number': '3G - Pending Activation Subscribers'})

# 8G

speed_8g = summary_df[((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Active'))&((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '8G/8G'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()
speed_8g = speed_8g.rename(columns = {'Order Number': '8G - Active Subscribers'})

speed_8g_pending = summary_df[((summary_df['Due Date'] == 0)| \
                                 ((pd.to_datetime(summary_df['Due Date']).dt.date > yesterday_date) & (pd.to_datetime(summary_df['Due Date']).dt.date <= future_date))) \
                              &((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Pending Activation')) \
                              &((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '8G/8G'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()

speed_8g_pending = speed_8g_pending.rename(columns = {'Order Number': '8G - Pending Activation Subscribers'})

In [61]:
final_summary = enablement_base_state.merge(speed_8g_pending.merge(speed_8g.merge(speed_3g_pending.merge((speed_3g.merge((speed_1g_pending.merge((speed_1g.merge((speed_200m_pending.merge((speed_200m.merge((speed_940m.merge(speed_940m_pending, how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level']), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level']), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level']), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])


In [62]:
final_summary = final_summary.fillna(0)

In [63]:
# final_summary['Aggregate 200M Penetration Rate'] = final_summary['200M - Active Subscribers']/final_summary['Enabled_LUs_WireCenter_Level']
# final_summary['Aggregate 940M Penetration Rate'] = final_summary['940M - Active Subscribers']/final_summary['Enabled_LUs_WireCenter_Level']
# final_summary['Aggregate 1G Penetration Rate'] = final_summary['1G - Active Subscribers']/final_summary['Enabled_LUs_WireCenter_Level']
# final_summary['Aggregate 3G Penetration Rate'] = final_summary['3G - Active Subscribers']/final_summary['Enabled_LUs_WireCenter_Level']

In [64]:
final_summary['Total Active Subscribers'] = final_summary['8G - Active Subscribers'] + final_summary['3G - Active Subscribers'] + final_summary['1G - Active Subscribers'] + final_summary['200M - Active Subscribers'] + final_summary['940M - Active Subscribers']

In [65]:
final_summary['Total Pending Activation Subscribers'] = final_summary['8G - Pending Activation Subscribers'] + final_summary['3G - Pending Activation Subscribers'] + final_summary['1G - Pending Activation Subscribers'] + final_summary['200M - Pending Activation Subscribers'] + final_summary['940M - Pending Activation Subscribers']

In [66]:
final_summary['Aggregate Penetration Rate (Active Subs)'] = final_summary['Total Active Subscribers'] / final_summary['Enabled_LUs_WireCenter_Level']

In [67]:
# uncomment to check data
# final_summary[final_summary['Total Pending Activation Subscribers']>0]

In [68]:
# join wire center level mg enabled lu count

final_summary_mg = final_summary.merge(mg_lu_df, on = 'WIRE_CENTER_ID', how = 'left')
final_summary_mg['MG_ENABLED_LU_CNT'] = final_summary_mg['MG_ENABLED_LU_CNT'].fillna(0)
final_summary_mg = final_summary_mg.rename(columns = {'MG_ENABLED_LU_CNT': 'Multigig_Enabled_LUs'})

In [69]:
state_summary = final_summary_mg.groupby(['STATE'], as_index = False).sum()

In [70]:
state_summary = state_summary[['STATE','Enabled_LUs_WireCenter_Level','Multigig_Enabled_LUs','8G - Pending Activation Subscribers','8G - Active Subscribers','3G - Pending Activation Subscribers','3G - Active Subscribers','1G - Pending Activation Subscribers','1G - Active Subscribers','940M - Active Subscribers','940M - Pending Activation Subscribers','200M - Pending Activation Subscribers','200M - Active Subscribers','Total Active Subscribers','Total Pending Activation Subscribers']]

In [71]:
# add state_summary aggregate penetration rate

state_summary['Aggregate Penetration Rate (Active Subs)'] = state_summary['Total Active Subscribers']/state_summary['Enabled_LUs_WireCenter_Level']

In [72]:
state_summary

,STATE,Enabled_LUs_WireCenter_Level,Multigig_Enabled_LUs,8G - Pending Activation Subscribers,8G - Active Subscribers,3G - Pending Activation Subscribers,3G - Active Subscribers,1G - Pending Activation Subscribers,1G - Active Subscribers,940M - Active Subscribers,940M - Pending Activation Subscribers,200M - Pending Activation Subscribers,200M - Active Subscribers,Total Active Subscribers,Total Pending Activation Subscribers,Aggregate Penetration Rate (Active Subs)
0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,39.0,8.0,0.0,1.0,48.0,inf
1,AZ,13606.0,10138.0,0.0,0.0,0.0,1.0,26.0,21.0,106.0,11.0,11.0,53.0,181.0,48.0,0.013303
2,CO,7598.0,1212.0,0.0,0.0,0.0,1.0,1.0,31.0,55.0,11.0,9.0,32.0,119.0,21.0,0.015662
3,FL,22954.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.0,240.0,107.0,17.0,121.0,347.0,0.005271
4,MN,8418.0,8353.0,0.0,1.0,1.0,0.0,7.0,23.0,28.0,12.0,5.0,34.0,86.0,25.0,0.010216
5,OR,349.0,264.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.002865
6,UT,3080.0,241.0,0.0,0.0,0.0,0.0,1.0,0.0,14.0,16.0,8.0,9.0,23.0,25.0,0.007468
7,WA,2286.0,340.0,0.0,0.0,0.0,2.0,0.0,11.0,57.0,24.0,1.0,3.0,73.0,25.0,0.031934


In [73]:
# create MG state summary
# 10/12/22 - comment below out to allow null states to be counted
state_summary_mg = state_summary[(state_summary['Multigig_Enabled_LUs']>0) | (state_summary['STATE'] == 0)]

In [74]:
state_summary_mg = state_summary_mg.drop(['Aggregate Penetration Rate (Active Subs)','Enabled_LUs_WireCenter_Level','200M - Pending Activation Subscribers','200M - Active Subscribers','940M - Active Subscribers','940M - Pending Activation Subscribers'], axis = 1)

In [75]:
state_summary_mg['Total Active Subscribers'] = state_summary_mg['8G - Active Subscribers'] + state_summary_mg['3G - Active Subscribers'] + state_summary_mg['1G - Active Subscribers']

In [76]:
state_summary_mg['Total Pending Activation Subscribers'] = state_summary_mg['8G - Pending Activation Subscribers'] + state_summary_mg['3G - Pending Activation Subscribers'] + state_summary_mg['1G - Pending Activation Subscribers'] 

In [77]:
state_summary_mg['MG Penetration Rate (Active Subs)'] = state_summary_mg['Total Active Subscribers'] / state_summary_mg['Multigig_Enabled_LUs']

In [78]:
# reorder columns
columns1 = ['STATE', 'Multigig_Enabled_LUs', 'Total Active Subscribers',
       'Total Pending Activation Subscribers',
       'MG Penetration Rate (Active Subs)',
       '8G - Active Subscribers', '8G - Pending Activation Subscribers',
       '3G - Active Subscribers','3G - Pending Activation Subscribers', 
       '1G - Active Subscribers','1G - Pending Activation Subscribers']

state_summary_mg = state_summary_mg[columns1]

columns2 = ['STATE', 'Enabled_LUs_WireCenter_Level', 'Multigig_Enabled_LUs',
        'Total Active Subscribers', 'Total Pending Activation Subscribers',
       'Aggregate Penetration Rate (Active Subs)',
       '8G - Active Subscribers','8G - Pending Activation Subscribers', 
       '3G - Active Subscribers','3G - Pending Activation Subscribers', 
       '1G - Active Subscribers','1G - Pending Activation Subscribers', 
       '940M - Active Subscribers', '940M - Pending Activation Subscribers',
        '200M - Active Subscribers','200M - Pending Activation Subscribers']

state_summary = state_summary[columns2]

columns3 = ['STATE', 'Enabled_LUs_WireCenter_Level', 'Multigig_Enabled_LUs',
            '200M - Active Subscribers',
            '940M - Active Subscribers', 
            '1G - Active Subscribers',
            '3G - Active Subscribers',
            '8G - Active Subscribers',
            'Total Active Subscribers'
]

state_summary_view = state_summary[columns3]

In [79]:
# ready for export
state_summary_view = state_summary_view[state_summary_view['STATE'] != 0].sort_values(by = 'Enabled_LUs_WireCenter_Level', ascending = False)

In [81]:
import time

TodaysDate = time.strftime("%m%d%y")
excelfilename = "Output/" + TodaysDate +"_Salesforce_Summary_StateWCLevel.xlsx"

with pd.ExcelWriter(excelfilename) as writer:  
    state_summary.to_excel(writer, sheet_name='State_Summary', index = False)
    state_summary_mg.to_excel(writer, sheet_name='MultiGig_State_Summary', index = False)
    final_summary.to_excel(writer, sheet_name='Summary', index = False)
    sf_df.to_excel(writer, sheet_name='Details', index = False)




### Fill template

In [82]:
# check number of rows = 7 otherwise move grand total position in templates:
state_summary_view

,STATE,Enabled_LUs_WireCenter_Level,Multigig_Enabled_LUs,200M - Active Subscribers,940M - Active Subscribers,1G - Active Subscribers,3G - Active Subscribers,8G - Active Subscribers,Total Active Subscribers
3,FL,22954.0,0.0,17.0,104.0,0.0,0.0,0.0,121.0
1,AZ,13606.0,10138.0,53.0,106.0,21.0,1.0,0.0,181.0
4,MN,8418.0,8353.0,34.0,28.0,23.0,0.0,1.0,86.0
2,CO,7598.0,1212.0,32.0,55.0,31.0,1.0,0.0,119.0
6,UT,3080.0,241.0,9.0,14.0,0.0,0.0,0.0,23.0
7,WA,2286.0,340.0,3.0,57.0,11.0,2.0,0.0,73.0
5,OR,349.0,264.0,0.0,0.0,0.0,1.0,0.0,1.0


In [83]:
with pd.ExcelWriter('View/Multigig_Sales_View.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    state_summary_view.to_excel(writer, 
                      sheet_name="Sheet1", 
                      startrow=2, 
                      index=False, 
                      header=False) 

In [85]:
# create archived copy
import shutil

excelfilename = "View/archive/" + TodaysDate +"_Multigig_Sales_View.xlsx"
shutil.copyfile('View/Multigig_Sales_View.xlsx', excelfilename)

'View/archive/112822_Multigig_Sales_View.xlsx'